# Using ZhipuAI GLM-4 though OpenAI SDK and Ecosystem

**This tutorial is available in English and is attached below the Chinese explanation**

本节代码将带领大家使用`OpenAI`的 SDK 和支持的框架调用 `GLM-4` 模型
你将在本章节学习到如何使用 Langchain 的 `ChatOpenAI` 类来调用 ZhipuAI 的 GLM-4 模型，以及如何使用 `OpenAI SDK` 类来实现调用GLM-4模型。

This notebook section will walk you through calling `GLM-4` models using the `OpenAI` SDK and supporting frameworks.
In this section you will learn how to call ZhipuAI's GLM-4 models using Langchain's `ChatOpenAI` class, and how to use the `OpenAI SDK` class to implement calling GLM-4 models.


## 1. Use OpenAI SDK

首先，需要导入 `OpenAI` SDK，同时，修改 `OpenAI` 的 `base_url` 为 `https://open.bigmodel.cn/api/paas/v4/` 
以便调用 `GLM-4` 模型。完成更换后，仅需填写 Key 就能使用 `GLM-4` 模型。

First, you need to import the `OpenAI` SDK and change the `OpenAI` `base_url` to `https://open.bigmodel.cn/api/paas/v4/` in order to call the `GLM-4` model. 
in order to call the `GLM-4` model. Once you have done this, you can use the `GLM-4` model by simply filling in the Key.
``python

In [2]:
from openai import OpenAI

client = OpenAI(
    api_key="your api key",
    base_url="https://open.bigmodel.cn/api/paas/v4/")

## 2. Simple use of OpenAI SDK to call GLM-4

我将带领大家完成最基础的对话调用。
请注意：
- `temperature` 参数的区间为 (0,1)。
- `do_sample = False (temperature = 0)` 在 OpenAI 调用中并不适用。
-  异步操作不适用。

其他例子(例如工具调用)，也均可以按照OpenAI的方式进行调用。


I will lead everyone to complete the most basic dialogue calls.
Please note:

- The interval of the `temperature` parameter is (0,1).
- `do_sample=False (temperature=0)` is not applicable in OpenAI calls.
- Asynchronous operations are not applicable.

Other examples (such as tool calls) can also be called in the same way as OpenAI.

In [3]:
response = client.chat.completions.create(
    model="glm-4",
    messages=[
        {
            "role": "user",
            "content": "tell me a joke"
        }
    ],
    top_p=0.7,
    temperature=0.9,
    stream=False,
    max_tokens=2000,
)
response

ChatCompletion(id='8605595589064489203', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Sure! Here's a light-hearted joke for you:\n\nWhy don't scientists trust atoms?\n\nBecause they make up everything! 😄", role='assistant', function_call=None, tool_calls=None))], created=1714287444, model='glm-4', object=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=28, prompt_tokens=9, total_tokens=37), request_id='8605595589064489203')

## 3. Using Langchain's ChatOpenAI class

Langchain 的 `ChatOpenAI` 类是对 `OpenAI` SDK 的封装，可以更方便调用。这里展示了如何使用 `ChatOpenAI` 类来调用 `GLM-4` 模型。

Langchain's `ChatOpenAI` class is a wrapper around the `OpenAI` SDK, making it easier to call. Here we show how to use the `ChatOpenAI` class to call the `GLM-4` model.

In [4]:
import os
from langchain_openai import ChatOpenAI
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory

llm = ChatOpenAI(
    temperature=0.95,
    model="glm-4",
    openai_api_key="your api key",
    openai_api_base="https://open.bigmodel.cn/api/paas/v4/"
)
prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(
            "You are a nice chatbot having a conversation with a human."
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{question}")
    ]
)

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
conversation = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory
)
conversation.invoke({"question": "tell me a joke"})

/Users/zr/Code/glm-cookbook/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(




> Entering new LLMChain chain...
Prompt after formatting:
System: You are a nice chatbot having a conversation with a human.
Human: tell me a joke

> Finished chain.


{'question': 'tell me a joke',
 'chat_history': [HumanMessage(content='tell me a joke'),
  AIMessage(content="Why don't scientists trust atoms?\n\nBecause they make up everything! 😄")],
 'text': "Why don't scientists trust atoms?\n\nBecause they make up everything! 😄"}

## 4. Calling GLM-4 with LangChain AgentExecutor
同时，GLM-4也能很好的接入到 LangChain 的 AgentExecutor 中，这里展示了如何使用 `AgentExecutor` 来调用 `GLM-4` 模型。

GLM-4 also plugs nicely into LangChain's AgentExecutor, which shows how to call a `GLM-4` model using `AgentExecutor`.

In [5]:
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain_community.tools.tavily_search import TavilySearchResults

os.environ["TAVILY_API_KEY"] = "your tavily api key"
tools = [TavilySearchResults(max_results=2)]
prompt = hub.pull("hwchase17/react")
# Choose the LLM to use
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke({"input": "what is LangChain?"})



> Entering new AgentExecutor chain...
To answer this question, I should use a search engine to gather information about LangChain. Since it's a topic I'm not very familiar with, comprehensive and accurate results would be beneficial.

Action: tavily_search_results_json
Action Input: What is LangChain?
Observation[{'url': 'https://python.langchain.com/docs/get_started/introduction/', 'content': "Introduction. LangChain is a framework for developing applications powered by large language models (LLMs). LangChain simplifies every stage of the LLM application lifecycle: Development: Build your applications using LangChain's open-source building blocks and components. Hit the ground running using third-party integrations and Templates."}, {'url': 'https://python.langchain.com/docs/modules/agents/concepts/', 'content': 'Note that observation is currently left as type Any to be maximally flexible. In practice, this is often a string. ... LangChain provides a wide set of built-in tools, but 

{'input': 'what is LangChain?',
 'output': 'LangChain is a framework that aids in the development of applications using large language models by offering open-source building blocks, components, and templates to streamline the process.'}

## 5. Using OpenAI SDK to call CogView-3

我们也支持使用OpenAI接口调用 `CogView-3` 模型，这里展示了如何使用 `OpenAI SDK` 类来调用 `CogView-3` 模型。但是，仍然有很多参数是无法使用的。

We also support using the OpenAI interface to call the `CogView-3` model. Here we show how to use the `OpenAI SDK` class to call the `CogView-3` model. However, there are still many parameters that cannot be used.

In [6]:
response_cogview = client.images.generate(
    model="cogview-3",
    prompt="一个城市在水晶瓶中的场景",

    # these parameters are not available
    # n=2,
    # response_format="base64",
    # size="medium",
    # style="natural",
    # quality="hd",
)
response_cogview

ImagesResponse(created=1714287461, data=[Image(b64_json=None, revised_prompt=None, url='https://sfile.chatglm.cn/testpath/7020b0f4-cbbb-562c-8196-576122d6c167_0.png')])

In [7]:
from IPython.display import display, Image

display(Image(url=response_cogview.data[0].url))

## 6. UsingOpenAI SDK to call Embedding Model

我们也支持使用openAI接口调用 `Embedding` 模型，这里展示了如何使用 `OpenAI SDK` 类来调用 `Embedding` 模型。

请注意，`Embedding` 模型的输入暂不支持使用 列表，只能使用字符串。在这里，我仅仅展示了前10个值。

We also support using the OpenAI interface to call the `Embedding` model. Here we show how to use the `OpenAI SDK` class to call the `Embedding` model.

Please note that the input of the `Embedding` model does not currently support lists and can only be used as strings. Here, I only show the first 10 values.

In [8]:
response_embedding = client.embeddings.create(
    model="embedding-2",
    input="你好",
    
    # these parameters are not available
    # input=["你好", "你是谁"],
    # dimensions=1024,
    # encoding_format="float",
)
response_embedding.data[0].embedding[:10] # only show the first 10

[-0.02675454691052437,
 0.019060475751757622,
 0.006672845687717199,
 -0.023301372304558754,
 0.0015068714274093509,
 -0.04486510157585144,
 -0.04257429763674736,
 -0.006881380919367075,
 0.04148220270872116,
 0.06300002336502075]